In [ ]:
import sys, time
from datetime import datetime
import numpy as np
import ecal.core.core as ecal_core
from scipy.spatial.transform import Rotation as R, Slerp
from copy import copy
from ruckig import InputParameter, OutputParameter, Result, Ruckig
from ranger import Ranger, Claw
from ranger import *
from ws_world import World
import threading
from xomnihand import XOmniHandGym

gym = XOmniHandGym()

ws_root = r"D:\bitBucket\rhea_ws"
sys.path += [ws_root]

from common.utils_py import *

tool = Pose.fromP6Vec([-190, 0, 0, 0, 0, 0]).homo

rg.mount()

world = World(ws_root)
world.listenWeight()

claw = Claw()

In [ ]:
###lambda poses
#让
ppp00 = Pose.fromP6(189.90, -19.47, 49.51,  -1.6287, 1.4562, -1.6354)  #对闸口
ppp01 = Pose.fromP6(190.34, -19.47, 36.70,  -1.6320, 1.4623, -1.6317)  #x,z最小, 避让上下料
#舀
pp0 = Pose.fromP6(200.66, -25.95, 55.89,  -1.6265, 1.4519, -1.6332)
pp1 = Pose.fromP6(225.05, -21.11, 55.89,  -1.6262, 1.4518, -1.4706)

pp5 = Pose.fromP6(256.27, -10.47, 55.89,  -1.6265, 1.4521, -1.4049)
pp6 = Pose.fromP6(258.65, -12.39, 55.89,  -1.6266, 1.4520, -1.4465)
pp7 = Pose.fromP6(259.37, -15.11, 55.89,  -1.6264, 1.4520, -1.4988)

pp8 = Pose.fromP6(259.37, -19.23, 55.89,  -1.6265, 1.4520, -1.5659)   
pp9 = Pose.fromP6(259.37, -19.97, 55.89,  -1.6264, 1.4520, -1.5838)  #打平
pp10 = Pose.fromP6(247.57, -21.17, 55.89,  -1.6264, 1.4520, -1.5838) #调整点
pp100 = Pose.fromP6(196.90, -21.17, 55.89,  -1.6264, 1.4520, -1.5838) #瓶口水平

#倒
pp11 = Pose.fromP6(196.47, -13.01, 56.66,  -1.6260, 1.4517, -1.6019)
pp12 = Pose.fromP6(195.27, -12.00, 56.52,  -1.6255, 1.4513, -1.4399)


###elite poses
#源试管托盘
ps0 = Pose.fromP6(52.62, 292.07, 158.46,  -3.1385, -0.0038, -1.5771)   #试管0夹持点位   
ps1 = Pose.fromP6(52.62, 251.83, 158.46,  -3.1385, -0.0038, -1.5771)   #试管1夹持点位
ps2 = Pose.fromP6(93.21, 292.08, 158.46,  -3.1385, -0.0038, -1.5771)   #试管2夹持点位
ps3 = Pose.fromP6(131.69, 292.08, 158.46,  -3.1385, -0.0038, -1.5771)   #试管3夹持点位

safe_pose = Pose.fromP6(121.66, 247.43, 241.69,  -3.1384, -0.0038, -0.6534)


#源目标试管放置
pm1 = Pose.fromP6(107.47, 294.81, 227.82,  3.1404, 0.5811, -1.5785)
pm2 = Pose.fromP6(159.85, 294.81, 176.95,  3.1370, 0.8703, -1.5807)
pm3 = Pose.fromP6(173.12, 321.41, 92.19,  3.1213, 1.3043, -1.5949)

#目标试管放置点
po1 = Pose.fromP6(173.64, 340.68, 60.20,  3.1181, 1.3356, -1.5980)   


##源试管放置
pm11 = Pose.fromP6(330.37, 314.19, 235.95,  -3.1384, -0.0038, -0.0177)
#目标试管孔上   降36.5，松
po2 = Pose.fromP6(330.33, 360.54, 222.31,  -3.1384, -0.0037, -0.0177) 


In [ ]:
class DiffList:
    def __init__(self):
        self.diff_list = [0 for i in range(10)]

    def update(self, value):
        self.diff_list.pop()
        self.diff_list.insert(0, value)


    def average(self):
        the_sum = 0
        for v in self.diff_list:
            the_sum += v
        return the_sum/len(self.diff_list)
    
    def __str__(self) -> str:
        return f"{[v for v in self.diff_list]}"


#####上下料#####
def take_src_tube():
    ##取源试管
    world.movePoseDelta(pose=ps0, z=50, line=1, speed=50)
    world.movePoseDelta(pose=ps0, z=0, line=1)
    claw.move(20)
    world.movePoseDelta(pose=ps0, z=50, line=1)
    world.movePose(safe_pose, line=1, speed=50)


def put_src_tube():
    ##进笼放置源试管
    world.movePose(pm1)
    world.movePose(pm2)
    world.movePose(pm3)
    world.movePose(po1)

    ##松开放置试管
    claw.move(250)
    time.sleep(0.5)
    claw.move(350)
    time.sleep(0.5)
    claw.move(450)
    world.movePoseDelta(pose=po1, z=-5, line=1, speed=15)

    ##退出
    world.movePoseDelta(pose=po1, y=-25, z=-5, line=1)
    world.movePose(pm2)
    world.movePose(pm1)
    world.movePose(safe_pose, line=1, speed=50)


def take_dst_tube(pose):
    ##取目标试管
    world.movePoseDelta(pose=pose, z=50, line=1, speed=50)
    world.movePoseDelta(pose=pose, z=0, line=1)
    claw.move(20)
    world.movePoseDelta(pose=pose, z=50, line=1)
    world.movePose(safe_pose, line=1, speed=50)


def put_dst_tube():
    ##放目标试管
    world.movePose(pm11, line=1, speed=50)
    world.movePose(po2, line=1, speed=50)
    world.movePoseDelta(pose=po2, z=-36.0)
    claw.move(250)
    world.movePoseDelta(pose=po2)
    world.movePose(pm11, line=1, speed=50)
    world.movePose(safe_pose, line=1, speed=50)


def remove_dst_tube():
    ##移除目标试管
    world.movePose(pm11, line=1, speed=50)
    world.movePose(po2, line=1, speed=50)
    world.movePoseDelta(pose=po2, z=-38.0)
    claw.move(20)
    world.movePoseDelta(pose=po2)
    world.movePose(pm11, line=1, speed=50)
    world.movePose(safe_pose, line=1, speed=50)


def reback_dst_tube(pose):
    ##放回目标试管
    world.movePoseDelta(pose=pose, z=50, line=1, speed=50)
    world.movePoseDelta(pose=pose, z=0, line=1)
    claw.move(240)
    world.movePoseDelta(pose=pose, z=50, line=1)
    world.movePose(safe_pose, line=1, speed=50)


def remove_src_tube():
    world.movePose(safe_pose, line=1, speed=50)
    world.movePose(pm1, line=1)
    world.movePose(pm2, line=1)
    world.movePoseDelta(pose=po1, y=-25, z=-5, line=1)
    claw.move(350)
    world.movePoseDelta(pose=po1, z=-5, line=1)
    claw.move(20)
    world.movePose(pm3)
    world.movePose(pm2)
    world.movePose(pm1)
    world.movePose(safe_pose, line=1, speed=50)


def reback_src_tube():
    ##取源试管
    world.movePose(safe_pose, line=1, speed=50)
    world.movePoseDelta(pose=ps0, z=50, line=1)
    world.movePoseDelta(pose=ps0, z=0, line=1)
    claw.move(240)
    world.movePoseDelta(pose=ps0, z=50, speed=50)
    world.movePose(safe_pose, line=1, speed=50)


def check_lambda():
    if not (48 < rg.getPose().toP6()[2] < 66):
        raise Exception("lambda is out the grid.")


#####笼端#####
# 离开称盘
# lift_move(17200)
# 完全脱离托杆
# lift_move(10200)
# 不影响回零传感器的最小值
# turn_move(20)
# 关盖状态极限位
# turn_move(120)
weight_connect_h = 20200
weight_release_h = 30800
prepare_h = 40200
prepare_r = 1200
spoon_h = 31800
spoon_r = 130

def prepare():
    # 初始状态
    lift_move(prepare_h)
    turn_move(prepare_r)
    time.sleep(5)
    rg.movePose(ppp01)


def lock_src_tube():
    # 压试管
    check_lambda()
    lift_move(spoon_h)  #
    turn_move(spoon_r)  #


def get_ev_weight():
    rg.log(rg.now(), "get ev weight start")
    sum = 0.0
    times = 4
    for i in range(times):
        time.sleep(0.5)
        #print(i, world.crt_weight)
        sum += world.crt_weight * 1000
    world.ev_weight = sum / times
    #print("ev:", world.ev_weight)
    rg.log(rg.now(), "get ev weight  end, ", f"reading this: {round(world.ev_weight, 2)}")
    return round(world.ev_weight, 2)


def deltaPose(pose, toolMatrix=tool, dx=0, dy=0, dz=0):
    dp = Pose.fromP6(dx, dy, dz, 0, 0, 0).homo
    return Pose(pose.homo @ toolMatrix @ dp @ np.linalg.inv(toolMatrix))


#试管中 dz 进出，dx 左右，dy 上下

def path_planing(dx=0, dz=0, max_v=20.0, max_a=10.0, max_j=100.0):
    rg.log(rg.now(), sys._getframe().f_code.co_name, " start: ", f"dx: {dx}, dz: {dz}, max_v: {max_v}, max_a: {max_a}, max_j: {max_j}")

    dpp5 = deltaPose(pp5, dx=dx, dz=dz)
    dpp6 = deltaPose(pp6, dx=dx, dz=dz)
    dpp7 = deltaPose(pp7, dx=dx, dz=dz)
    dpp8 = deltaPose(pp8, dx=dx, dz=dz)
    dpp9 = deltaPose(pp9, dx=dx, dz=dz)

    points = [pp1]
    points += slerp_pose(dpp5, dpp6)
    points += slerp_pose(dpp6, dpp7)[1:]
    points += slerp_pose(dpp7, dpp8)[1:]
    points += slerp_pose(dpp8, dpp9)[1:]

    scale = np.array([[1., 1., 1., 1000.,],
                    [1., 1., 1., 1000.,],
                    [1., 1., 1., 1000.,],
                    [0., 0., 0., 1.]])

    joints = [gym.ik(p.homo * scale) for p in points]
    len(joints)

    steps, time = auto_planning(joints, max_v=max_v, max_a=max_a, max_j=max_j)

    rg.log(rg.now(), sys._getframe().f_code.co_name, " end")
    return steps, time



In [ ]:
rg.movePose(pp0)

In [ ]:
rg.movePose(deltaPose(pp0, dx=5))

In [ ]:
rg.movePose(ppp00)

In [ ]:
rg.movePose(ppp00)
lift_move(prepare_h)  #
turn_move(prepare_r)  #

In [ ]:
lift_move(spoon_h)  #
turn_move(spoon_r)  #

In [ ]:
turn_move(prepare_r)  #

In [ ]:
lift_move(prepare_h) 

In [ ]:
lift_move(weight_connect_h)

In [ ]:
lift_move(10000)

In [ ]:
prepare()

In [ ]:
rg.movePose(ppp00)
lock_src_tube()

In [ ]:
rg.movePose(pp0)


In [ ]:
rg.movePose(pp1)

In [ ]:
rg.movePose(deltaPose(pp5, dz=-2))

In [ ]:
##上料
# prepare()

# take_src_tube()
# put_src_tube()

# take_dst_tube(ps2)
# put_dst_tube()

# rg.movePose(ppp00)
# lock_src_tube()

In [ ]:
p_p = 0.3
p_d = 0.1
precision = 0.5

wround = 0
mt = 3
mo = []
ms = []
ma = []
mbf = []
maf = []
mass_in_dst = [0]
try_spoon = False

spoon_shift_z = -5
spoon_shift_x = 0

last_orignal_spooned = 0

In [ ]:
##前称量
def weight_before_start(wround):
    rg.log(rg.now(), sys._getframe().f_code.co_name, " start")
    rg.log_cut_begin(world)
    global mo
    if len(mo) != wround:
        print("wround and list len mismatch--", wround, len(mo))
        # raise Exception("wround error.")
    lift_move(weight_connect_h)
    rg.log(rg.now(), "waiting for weight to be stable start")
    time.sleep(5)
    rg.log(rg.now(), "waiting for weight to be stable end")
    mo.append(get_ev_weight())
    rg.log_cut_end(world)
    rg.log(rg.now(), sys._getframe().f_code.co_name, " end. ", "mo: ", str(mo))


##舀粉
def spoon_move(dx=0, dz=0):
    rg.log(rg.now(), sys._getframe().f_code.co_name, " start: ", f"dx: {dx}, dz: {dz}")

    rg.log_cut_begin(world)
    lift_move(spoon_h)
    rg.log(rg.now(), "path start")
    rg.movePose(pp0)
    rg.movePose(pp1)
    # 试管中 dz 进出，dx 左右，dy 上下
    steps, time_cost = path_planing(dx=dx, dz=dz)

    rg.log(rg.now(), "trajMove start")
    rc.call(
        "/dev/chl0/canopen!node_dev_zcan_rs:trajMove",
        {
            "jointsList": steps,
        },
    )
    time.sleep(time_cost + 1)
    rg.log(rg.now(), "trajMove end")

    rg.movePose(pp10, speed=2, acc=1)
    rg.log(rg.now(), "path end")
    rg.log_cut_end(world)

    rg.log(rg.now(), sys._getframe().f_code.co_name, " end")


##减重称
def weight_after_spoon(wround):
    rg.log(rg.now(), sys._getframe().f_code.co_name, " start")
    rg.log_cut_begin(world)

    global ms, mo
    lift_move(weight_connect_h)
    rg.log(rg.now(), "waiting for weight to be stable start")
    time.sleep(5)
    rg.log(rg.now(), "waiting for weight to be stable end")
    ms.append(get_ev_weight())

    rg.log_cut_end(world)
    last_orignal_spooned = mo[wround]-ms[wround]

    rg.log(
        rg.now(),
        sys._getframe().f_code.co_name,
        " end. ",
        f"ms: {ms}, spooned this: {last_orignal_spooned}",
    )


##调整
def adjust_spoon_auto():
    rg.log(rg.now(), sys._getframe().f_code.co_name, " start")
    rg.log_cut_begin(world)

    global mo, ms, wround, mass_in_dst, precision, try_spoon
    shake(rg, round=8, d=0.02, mode=1)
    time.sleep(1)

    try_count = 0
    escape_time = 0
    movej6_count = 0

    start_spooned_mass = round(mo[wround] - get_ev_weight(), 2)
    spooned_mass = start_spooned_mass
    gap_mass = mt - mass_in_dst[wround]

    crt_diff = spooned_mass - gap_mass
    diff_list = DiffList()
    diff_list.update(crt_diff)
    new_d = 0.02
    rg.log(
        rg.now(),
        "adjust begin status:",
        f"wround: {wround}, mass_in_dst: {mass_in_dst}, spooned_mass: {spooned_mass}, gap_mass: {gap_mass}",
    )

    while spooned_mass > gap_mass + precision:
        if new_d > 0.2:
            rg.log("@@ the shake amplitude is out limit!!")
            break
        shake(rg, round=8, d=new_d, mode=1)

        escape_time += 1
        spooned_mass = round(mo[wround] - get_ev_weight(), 2)
        crt_diff = spooned_mass - gap_mass
        new_d = (
            p_p * (spooned_mass - gap_mass) + p_d * diff_list.average() * escape_time
        ) / start_spooned_mass / 50.0 + 0.02
        diff_list.update(crt_diff)

        print(
            f"adjust shake ---, {escape_time:<4}, {spooned_mass:<4}, {round(crt_diff, 4):<6}, {round(p_d * diff_list.average() * escape_time, 4):<6}, {round(new_d, 4):<6}"
        )

        if escape_time > 40:
            rg.log(rg.now(), "shake out 40 times, try adjust j6.")
            escape_time = 1
            try_count += 1
            rg.moveJ6(3)
            rg.log(rg.now(), f"adjust j6 offset: {3*try_count} done.")

            if try_count > 10:
                try_spoon = True
                rg.log(rg.now(), f"try too many times {try_count*escape_time}, reset this round {wround}.")
                break

    if try_count > 0:
        rg.moveJ6(-3 * try_count, speed=20)
        spooned_mass = round(mo[wround] - get_ev_weight(), 2)

    ma.append(round(get_ev_weight(), 2))
    rg.log_cut_end(world)
    rg.log(rg.now(), sys._getframe().f_code.co_name, " end. ", f"ma: {ma}")


##抖落，增重称
# 退出，降低
def weight_before_fall(wround):
    rg.log(rg.now(), sys._getframe().f_code.co_name, " start")
    rg.log_cut_begin(world)

    global mbf
    rg.movePose(pp100, speed=3, acc=1)
    mbf.append(get_ev_weight())

    rg.log_cut_end(world)
    rg.log(rg.now(), sys._getframe().f_code.co_name, " end. ", f"mbf: {mbf}")


# 到目标瓶倒粉
def powder_fall(wround):
    rg.log(rg.now(), sys._getframe().f_code.co_name, " start")
    rg.log_cut_begin(world)

    global maf
    rg.movePose(pp11, line=True, speed=1, acc=1)
    # rg.movePose(pp12, line=True, speed=2, acc=1)
    rg.moveJ6(-90)
    shake(rg)
    rg.movePose(pp11)

    maf.append(get_ev_weight())
    rg.movePose(pp0)

    rg.log_cut_end(world)
    rg.log(rg.now(), sys._getframe().f_code.co_name, " end. ", f"maf: {maf}")


def update():
    global mo, ms, mbf, maf, wround, mass_in_dst, try_spoon, last_orignal_spooned, spoon_shift_z
    if last_orignal_spooned < mt-mass_in_dst[wround] and spoon_shift_z < 0:
        spoon_shift_z += 1
        rg.log(rg.now(), "adjust spoon offset +1")

    if try_spoon:
        mo.pop()
        ms.pop()
        try_spoon = False
    else:
        tmp = mass_in_dst[wround]
        mass_in_dst.append(round(maf[wround] - mbf[wround] + tmp, 2))
        wround += 1

    rg.log(rg.now(), "update:", f"wrond: {wround}, mass_in_dst: {mass_in_dst}")


In [ ]:
wround = 0
mt = 50
mo = []
ms = []
ma = []
mbf = []
maf = []
mass_in_dst = [0]
try_spoon = False

precision = 0.5
rg._log = []


In [ ]:
spoon_shift_z = -5
spoon_shift_x = 0

In [ ]:
for i in range(5):
    log_name = "l-phenylalaninie"

    while True:
        
        if mt-precision <= mass_in_dst[-1] <= mt+precision:
            rg.log("dispensing done.")
            break
        elif mass_in_dst[-1] > mt+precision:
            rg.log("the dst mass is more than need.")
            break
        
        weight_before_start(wround)

        spoon_move(dz=spoon_shift_z, dx=spoon_shift_x)

        weight_after_spoon(wround)

        adjust_spoon_auto()

        weight_before_fall(wround)

        powder_fall(wround)
    
        update()
        
    rg.log_save(log_name)
    
    wround = 0
    mo = []
    ms = []
    ma = []
    mbf = []
    maf = []
    mass_in_dst = [0]

    

In [ ]:
mass_in_dst

In [ ]:
rg.movePose(ppp00)
lift_move(prepare_h)  #
turn_move(prepare_r)  #

In [ ]:
lift_move(spoon_h)  #
turn_move(spoon_r)  #

In [ ]:
lift_move(10000)